A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [286]:
# Start your code here!
import pandas as pd

ds_jobs = pd.read_csv('customer_train.csv')
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0


In [287]:
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

In [288]:
ds_jobs.memory_usage()

Index                        128
student_id                153264
city                      153264
city_development_index    153264
gender                    153264
relevant_experience       153264
enrolled_university       153264
education_level           153264
major_discipline          153264
experience                153264
company_size              153264
company_type              153264
last_new_job              153264
training_hours            153264
job_change                153264
dtype: int64

In [289]:
# reset point 
df_new = ds_jobs.copy()
df_new.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0


In [290]:
# Convert integer columns to int32
df_new[[ 'student_id','training_hours']] = df_new[['student_id','training_hours']].astype('int32')

# Convert float columns to float16
df_new[['city_development_index']] = df_new[['city_development_index']].astype('float16')

df_new[['job_change']] = df_new[['job_change']].astype('int32')


In [291]:
# Update job_change column to remap numbers to text
# df_new['job_change'] = df_new['job_change'].map({0: 'No', 1: 'Yes'})

df_new.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.919922,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.775879,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624023,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789062,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767090,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0


In [292]:
# Unique values of 'company_size', 'experience', 'last_new_job'
unique_company_size = df_new['company_size'].unique()
unique_experience = df_new['experience'].unique()
unique_last_new_job = df_new['last_new_job'].unique()
unique_relevant_experience = df_new['relevant_experience'].unique()
unique_enrolled_university = df_new['enrolled_university'].unique()
unique_education_level = df_new['education_level'].unique()

unique_company_size, unique_experience, unique_last_new_job, unique_relevant_experience,unique_enrolled_university,unique_education_level

(array([nan, '50-99', '<10', '10000+', '5000-9999', '1000-4999', '10-49',
        '100-499', '500-999'], dtype=object),
 array(['>20', '15', '5', '<1', '11', '13', '7', '17', '2', '16', '1', '4',
        '10', '14', '18', '19', '12', '3', '6', '9', '8', '20', nan],
       dtype=object),
 array(['1', '>4', 'never', '4', '3', '2', nan], dtype=object),
 array(['Has relevant experience', 'No relevant experience'], dtype=object),
 array(['no_enrollment', 'Full time course', nan, 'Part time course'],
       dtype=object),
 array(['Graduate', 'Masters', 'High School', nan, 'Phd', 'Primary School'],
       dtype=object))

In [293]:
# unordered categories
df_new[['gender','city','company_type','enrolled_university','education_level','major_discipline']]= df_new[['gender','city','company_type','enrolled_university','education_level','major_discipline']].astype('category');

# Define the unique ordered categories for each column
ordered_company_size = ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+']
ordered_experience = ['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20']
ordered_last_new_job = ['never', '<1', '1', '2', '3', '4', '>4']
ordered_job_change = [0, 1]
ordered_relevant_exp = ['No relevant experience','Has relevant experience']
ordered_enrolled_university = ['no_enrollment', 'Part time course','Full time course' ]
ordered_education_level = ['Primary School','High School','Graduate',  'Masters', 'Phd' ]
# Convert the columns to ordered categorical data type
df_new['company_size'] = pd.Categorical(df_new['company_size'], categories=ordered_company_size, ordered=True)
df_new['experience'] = pd.Categorical(df_new['experience'], categories=ordered_experience, ordered=True)
df_new['last_new_job'] = pd.Categorical(df_new['last_new_job'], categories=ordered_last_new_job, ordered=True)
df_new['relevant_experience'] = pd.Categorical(df_new['relevant_experience'], categories=ordered_relevant_exp, ordered=True)
df_new['enrolled_university'] = pd.Categorical(df_new['enrolled_university'], categories=ordered_enrolled_university, ordered=True)
df_new['education_level'] = pd.Categorical(df_new['education_level'], categories=ordered_education_level, ordered=True)


df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  category
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            13220 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  category
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  int32   
dtypes: category(10), float

In [294]:
ds_jobs_clean = df_new.copy()
ds_jobs_clean.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.919922,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.775879,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624023,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789062,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767090,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0


In [295]:
students_with_10Plus = ds_jobs_clean['experience'] >= '10'
comapny_size_1000Plus = ds_jobs_clean['company_size'] >= '1000-4999'

ds_jobs_clean= ds_jobs_clean[comapny_size_1000Plus & students_with_10Plus]

ds_jobs_clean.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,NaN,Has relevant experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0
12,25619,city_61,0.913086,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,0
31,22293,city_103,0.919922,Male,Has relevant experience,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,0
34,26494,city_16,0.910156,Male,Has relevant experience,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,0
40,2547,city_114,0.925781,Female,Has relevant experience,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,0


In [296]:
ds_jobs_clean.memory_usage()

Index                     17608
student_id                 8804
city                       7353
city_development_index     4402
gender                     2333
relevant_experience        2325
enrolled_university        2333
education_level            2413
major_discipline           2421
experience                 2933
company_size               2565
company_type               2421
last_new_job               2557
training_hours             8804
job_change                 8804
dtype: int64